In [80]:
# Feature Engineering: imports
import warnings
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)

# Feature Engineering Notes (from EDA)

This markdown is a **data-quality + leakage checklist** based on what we observed in the EDA notebook. Fix/handle these issues before building features so your model is valid and stable.

## 1) Target + leakage risk (must handle)
- **`Customer Status` is the label** (Stayed/Churned). Do not include it as an input feature.
- **`Churn Category` and `Churn Reason` are leakage columns**: in EDA they were only populated for churned customers (many missing for non‑churn). These fields represent information known *after* churn, so using them will inflate performance unrealistically. **Drop them from training features.**

## 2) Missing values are not random (need domain-aware handling)
From the EDA `info()` / missingness counts:
- **`Offer` has a lot of missing values** (only ~3166 non-null out of 7043).
- **`Internet Type` has missing values** (only ~5517 non-null).

What this usually means in telecom churn data: missing values often indicate “not applicable” (e.g., customer has no internet service / no offer), not truly unknown.
- Prefer **explicit categories** like `No Offer` / `No Internet` instead of leaving NaN.
- When a feature is conditional on another feature (e.g., `Internet Type` depends on `Internet Service`), fill missing using those logical rules rather than using a generic mode imputation.

## 3) Wrong dtype / “numeric-but-categorical” fields
- **`Zip Code` was read as numeric (`int64`) in EDA**. Treat it as a **string/categorical** feature (or drop/group it) so you don’t create fake numeric relationships like “zip 90001 > 10001”.
- IDs/locations (e.g., `Customer ID`, `City`) should **not** be treated as numeric features even if they look like numbers.

## 4) High-cardinality categorical columns (encoding risk)
- **`City`** is high-cardinality (many unique values). One-hot encoding it can explode feature space and overfit.
  - Typical options: drop it, keep only top‑K cities and map the rest to `Other`, or engineer broader geo features (region/state) if available.
- `Customer ID` is effectively unique → **drop from modeling** (keep only for joins).

## 5) Redundant / highly correlated numeric features
EDA correlation analysis showed some numeric features are highly correlated (common in datasets containing totals + components).
- Example pattern: `Total Charges`, `Total Revenue`, `Monthly Charges`, `Tenure in Months` often encode the same information.
- During feature engineering, **avoid keeping multiple near-duplicates**; pick the most interpretable ones or create a single derived feature (e.g., `revenue_per_month`).

## 6) General quality checks to apply before feature engineering
- **Duplicates**: confirm there are no duplicate rows / duplicate `Customer ID`. If duplicates exist, decide whether to keep the latest record or aggregate.
- **String consistency**: strip whitespace, normalize Yes/No values, and standardize category casing (e.g., `"Yes"`, `" yes "`, `"YES"`).
- **Impossible values / outliers**: check for negative charges, extreme values, and inconsistent combinations (e.g., internet-related fields filled when `Internet Service == 'No'`). Handle via rules, capping, or investigation.

## What I will do in this notebook
1. Load raw data and create a clean, modeling-ready dataframe (drop leakage columns, fix dtypes, handle missingness logically).
2. Build a reproducible preprocessing pipeline using `ColumnTransformer` + `Pipeline` (imputers, scaling, encoding).
3. Save the processed dataset to `data/processed/` for modeling.

In [81]:
df= pd.read_csv("../data/raw/telecom_customer_churn.csv")
df_copy = df.copy()

In [82]:
df.head()

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,9,NaN,Yes,42.39,No,Yes,Cable,16.0,No,Yes,No,Yes,Yes,No,No,Yes,One Year,Yes,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,9,NaN,Yes,10.69,Yes,Yes,Cable,10.0,No,No,No,No,No,Yes,Yes,No,Month-to-Month,No,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,4,Offer E,Yes,33.65,No,Yes,Fiber Optic,30.0,No,No,Yes,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,13,Offer D,Yes,27.82,No,Yes,Fiber Optic,4.0,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,3,NaN,Yes,7.38,No,Yes,Fiber Optic,11.0,No,No,No,Yes,Yes,No,No,Yes,Month-to-Month,Yes,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [83]:
df.columns

Index(['Customer ID', 'Gender', 'Age', 'Married', 'Number of Dependents',
       'City', 'Zip Code', 'Latitude', 'Longitude', 'Number of Referrals',
       'Tenure in Months', 'Offer', 'Phone Service',
       'Avg Monthly Long Distance Charges', 'Multiple Lines',
       'Internet Service', 'Internet Type', 'Avg Monthly GB Download',
       'Online Security', 'Online Backup', 'Device Protection Plan',
       'Premium Tech Support', 'Streaming TV', 'Streaming Movies',
       'Streaming Music', 'Unlimited Data', 'Contract', 'Paperless Billing',
       'Payment Method', 'Monthly Charge', 'Total Charges', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges',
       'Total Revenue', 'Customer Status', 'Churn Category', 'Churn Reason'],
      dtype='object')

The internet service status indicates whether the customer is subscribed to internet service with the company, denoted as "Yes" or "No."

The total number of customers subscribing to internet services is 5,517.

As depicted in the preceding visualization, certain features are contingent upon internet service subscription. In other words, these features become accessible when the customer subscribes to internet service. These features include:

Internet Type
Online Security
Online Backup
Device Protection Plan
Premium Tech Support
Streaming TV
Streaming Movies
Streaming Music
Unlimited Data
Based on the aforementioned findings, it's evident that customers who don't subscribe to internet services have null values in the mentioned features. Consequently, to standardize these features for analysis, the null values within these columns should be replaced with "No."

In [84]:
#filling null values
df['Internet Type'] = df['Internet Type'].apply(lambda x: 'None' if pd.isnull(x) else x)
df['Offer'] = df['Offer'].apply(lambda x: 'None' if pd.isnull(x) else x)

column_name = ['Online Security', 'Online Backup', 'Device Protection Plan', 'Premium Tech Support',
               'Streaming TV', 'Streaming Movies', 'Streaming Music', 'Unlimited Data']
for column in column_name:
  df[column] = df[column].apply(lambda x: 'No' if pd.isnull(x) else x)

In [85]:
df.drop(columns=['Customer ID','Total Charges'],inplace=True)

In [86]:
# first step:
conversion_dict = {"Yes": 1, "No": 0}
df['Multiple Lines'] = df['Multiple Lines'].map(conversion_dict)

# second step: labeling the target by using LabelEncoder
label_encoder = LabelEncoder()
df['Customer Status'] = label_encoder.fit_transform(df['Customer Status'])


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             7043 non-null   object 
 1   Age                                7043 non-null   int64  
 2   Married                            7043 non-null   object 
 3   Number of Dependents               7043 non-null   int64  
 4   City                               7043 non-null   object 
 5   Zip Code                           7043 non-null   int64  
 6   Latitude                           7043 non-null   float64
 7   Longitude                          7043 non-null   float64
 8   Number of Referrals                7043 non-null   int64  
 9   Tenure in Months                   7043 non-null   int64  
 10  Offer                              7043 non-null   object 
 11  Phone Service                      7043 non-null   objec

In [88]:
obj_col = df.select_dtypes(include="object").columns
obj_col

Index(['Gender', 'Married', 'City', 'Offer', 'Phone Service',
       'Internet Service', 'Internet Type', 'Online Security', 'Online Backup',
       'Device Protection Plan', 'Premium Tech Support', 'Streaming TV',
       'Streaming Movies', 'Streaming Music', 'Unlimited Data', 'Contract',
       'Paperless Billing', 'Payment Method', 'Churn Category',
       'Churn Reason'],
      dtype='object')

In [89]:
encoder = {}

for i in obj_col:
    label_encoder = LabelEncoder()
    df[i] = label_encoder.fit_transform(df[i])
    encoder[i] = label_encoder

In [90]:
encoder

{'Gender': LabelEncoder(),
 'Married': LabelEncoder(),
 'City': LabelEncoder(),
 'Offer': LabelEncoder(),
 'Phone Service': LabelEncoder(),
 'Internet Service': LabelEncoder(),
 'Internet Type': LabelEncoder(),
 'Online Security': LabelEncoder(),
 'Online Backup': LabelEncoder(),
 'Device Protection Plan': LabelEncoder(),
 'Premium Tech Support': LabelEncoder(),
 'Streaming TV': LabelEncoder(),
 'Streaming Movies': LabelEncoder(),
 'Streaming Music': LabelEncoder(),
 'Unlimited Data': LabelEncoder(),
 'Contract': LabelEncoder(),
 'Paperless Billing': LabelEncoder(),
 'Payment Method': LabelEncoder(),
 'Churn Category': LabelEncoder(),
 'Churn Reason': LabelEncoder()}

In [91]:
label_column = df['Customer Status']
df.drop(columns=['Customer Status'], inplace=True)

imputer = SimpleImputer()
df_imputed = imputer.fit_transform(df)

df_imputed = pd.DataFrame(df_imputed, columns=df.columns)
df_imputed['Customer Status'] = label_column


In [92]:
df_imputed.head()

,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Churn Category,Churn Reason,Customer Status
0,0.0,37.0,1.0,0.0,346.0,93225.0,34.827662,-118.999073,2.0,9.0,0.0,1.0,42.39,0.0,1.0,0.0,16.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,65.6,0.00,0.0,381.51,974.81,5.0,20.0,2
1,1.0,46.0,0.0,0.0,368.0,91206.0,34.162515,-118.203869,0.0,9.0,0.0,1.0,10.69,1.0,1.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-4.0,38.33,10.0,96.21,610.28,5.0,20.0,2
2,1.0,50.0,0.0,0.0,222.0,92627.0,33.645672,-117.922613,0.0,4.0,5.0,1.0,33.65,0.0,1.0,2.0,30.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,73.9,0.00,0.0,134.60,415.45,1.0,2.0,0
3,1.0,78.0,1.0,0.0,587.0,94553.0,38.014457,-122.115432,1.0,13.0,4.0,1.0,27.82,0.0,1.0,2.0,4.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,98.0,0.00,0.0,361.66,1599.51,2.0,18.0,0
4,0.0,75.0,1.0,0.0,139.0,93010.0,34.227846,-119.079903,3.0,3.0,0.0,1.0,7.38,0.0,1.0,2.0,11.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,83.9,0.00,0.0,22.14,289.54,2.0,14.0,0


In [93]:
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             7043 non-null   float64
 1   Age                                7043 non-null   float64
 2   Married                            7043 non-null   float64
 3   Number of Dependents               7043 non-null   float64
 4   City                               7043 non-null   float64
 5   Zip Code                           7043 non-null   float64
 6   Latitude                           7043 non-null   float64
 7   Longitude                          7043 non-null   float64
 8   Number of Referrals                7043 non-null   float64
 9   Tenure in Months                   7043 non-null   float64
 10  Offer                              7043 non-null   float64
 11  Phone Service                      7043 non-null   float

In [94]:
df_imputed['Customer Status'].value_counts()

Customer Status
2    4720
0    1869
1     454
Name: count, dtype: int64

In [103]:
x = df_imputed.drop(columns=['Customer Status'])
y = df_imputed.iloc[:,-1]

In [104]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [106]:
def handle_outliers(X_train, X_test, columns):
    for col in columns:
        Q1 = np.percentile(X_train[col], 25)
        Q3 = np.percentile(X_train[col], 75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Handling outliers in X_train
        X_train[col] = np.where(X_train[col] < lower_bound, lower_bound, X_train[col])
        X_train[col] = np.where(X_train[col] > upper_bound, upper_bound, X_train[col])

        # Handling outliers in X_test based on X_train IQR
        X_test[col] = np.where(X_test[col] < lower_bound, lower_bound, X_test[col])
        X_test[col] = np.where(X_test[col] > upper_bound, upper_bound, X_test[col])

    return X_train, X_test


# Define the columns to apply outlier handling
columns_to_handle = ['Avg Monthly GB Download', 'Total Revenue', 'Number of Dependents',
                     'Total Refunds', 'Total Extra Data Charges']

# Apply the function to the specified columns
X_train, X_test = handle_outliers(X_train, X_test, columns_to_handle)

In [113]:
num_col = df.select_dtypes(include=['int64','float64']).columns
num_col

Index(['Age', 'Number of Dependents', 'Zip Code', 'Latitude', 'Longitude',
       'Number of Referrals', 'Tenure in Months',
       'Avg Monthly Long Distance Charges', 'Multiple Lines',
       'Avg Monthly GB Download', 'Monthly Charge', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges',
       'Total Revenue'],
      dtype='object')

In [114]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and test data
X_train[num_col] = scaler.fit_transform(X_train[num_col])
X_test[num_col] = scaler.transform(X_test[num_col])